Check bản ghi doanh so

In [9]:
from src.Get_data_DB import DataTransformer
# Tạo instance của class
transformer = DataTransformer()

# Lấy dữ liệu lần đầu từ năm 2024 Mysql
mysql_query = """SELECT 
                      tp.id AS Id,
                      s.money AS Doanh_so,
                      DATE_ADD(t.payment_at, INTERVAL 7 HOUR) AS Ngay_chuyen_tien,
                      s.name as Ten_khach,
                      s.email as Khach_hang,
                      s.mobile as So_dt,
                      p.code AS Ten_khoa_hoc,
                      o.current_sale_id AS Ma_saler,
                      l.utm_medium AS Ma_marketer,
                      o.source AS Ma_kenh
    FROM transactions t
    JOIN transactions_products tp ON tp.p_transaction_id = t.id
    JOIN products p ON tp.product_id = p.id
    JOIN orders o ON t.order_id = o.id
    join students s on s.order_id=o.id
    JOIN leads l ON o.lead_id = l.id
    join users u on o.current_sale_id=u.id
    WHERE t.t_status = 1
      AND DATE_ADD(t.payment_at, INTERVAL 7 HOUR) >= '2025-09-01' 
      and DATE_ADD(t.payment_at, INTERVAL 7 HOUR) <= '2025-09-16' 
      And u.role=3
#     and u.name= 'Tong Trung'
    GROUP BY tp.id, Ten_khoa_hoc, Ma_saler, Ma_marketer, Ma_kenh,Khach_hang,Doanh_so, Ten_khach, So_dt
                    """
df1 = transformer.fetch_from_mysql(mysql_query)
df1

,Id,Doanh_so,Ngay_chuyen_tien,Ten_khach,Khach_hang,So_dt,Ten_khoa_hoc,Ma_saler,Ma_marketer,Ma_kenh
0,15870,3900000,2025-09-03,Nguyễn Hải Long,tronyeulmht@gmail.com,0383349395,AI.Nocode,135,,1
1,15878,6000000,2025-09-05,Ly Tâm,vangbuihy@gmail.com,0966967238,DS.01,133,,0
2,15879,3500000,2025-09-05,Văn Hữu Hàng,vanhang.qn@gmail.com,0379221893,AI.Nocode,135,BinhND,0
3,15880,15000000,2025-09-06,Trần Trung Tín,sanbang38@gmail.com,0933911717,ML.I,65,BinhND,0
4,15881,13500000,2025-09-08,Nguyễn Đức,virtue.nguyenv@gmail.com,0961322509,ITBA,113,HienPT,0
5,15882,20990000,2025-09-08,Ngô Thùy,Ngovanthuy2006@gmail.com,0335343347,AI.2025,85,BinhND,0
6,15884,3900000,2025-09-07,Thành Đạt,qdat14062k2@gmail.com,0901993899,AI.Nocode,135,BinhND,0
7,15885,20990000,2025-09-08,Nguyen Trong Hieu,nth.hn1990@gmail.com,0372422582,DE.COMBO01,75,None,2
8,15886,8500000,2025-09-08,Nguyễn Ái Vy,nguyenaivy191020@gmail.com,0935565740,ITBA,65,HienPT,1
9,15887,20401361,2025-09-09,Trần Thị Vân Anh,tinatrantran@gmail.com,0909188906,AI.2025,85,HienPT,1
